# How to create a nested hierarchical mesh?

In [ ]:
import geomstats.backend as gs
from geomfum.dataset import NotebooksDataset
from geomfum.shape import TriangleMesh
from geomfum.shape.hierarchical import HierarchicalMesh, NestedHierarchicalMesh

[Load mesh](00_load_mesh_from_file.ipynb).

In [2]:
dataset = NotebooksDataset()
mesh = TriangleMesh.from_file(dataset.get_filename("cat-00"))

mesh.n_vertices, mesh.n_faces

INFO: Data has already been downloaded... using cached file ('C:\Users\giuli\.geomfum\data\cat-00.off').


(7207, 14410)

Create a nested [hierarchical mesh](./11_hierarchical_mesh.ipynb).

In [3]:
min_n_samples = [1000, 500, 250]

nested_hmesh = NestedHierarchicalMesh.from_hierarchical_shape(
    mesh, HierarchicalMesh.from_registry, min_n_samples=min_n_samples
)

nested_hmesh.n_vertices

C:\Users\giuli\OneDrive\Research\geomfum_proj\geomfum\geomfum\wrap\pyrmt.py:69: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:209.)
  return TriangleMesh(gs.asarray(rlow.vertices), gs.asarray(rlow.triangles))


[302, 505, 1004, 7207]

We can access each mesh from low to high resolution with `nested_hmesh.meshes`.

We can transfer scalars from low-resolution to high-resolution.

In [5]:
low_scalar = gs.random.uniform(size=nested_hmesh.lowest.n_vertices)

scalars = nested_hmesh.scalar_low_high(low_scalar)

[scalar_.shape for scalar_ in scalars]

[torch.Size([302]), (505,), (1004,), (7207,)]

Or stop at a particular level.

In [6]:
scalars = nested_hmesh.scalar_low_high(low_scalar, n_levels=2)

[scalar_.shape for scalar_ in scalars]

[torch.Size([302]), (505,), (1004,)]

With this, we can extend a [low-resolution basis](./02_mesh_laplacian_spectrum.ipynb) (as above, we can stop at a particular level).

In [7]:
nested_hmesh.lowest.laplacian.find_spectrum(spectrum_size=10, set_as_basis=True)

nested_hmesh.extend_basis(set_as_basis=True)

[mesh_.basis for mesh_ in nested_hmesh.meshes]

C:\Users\giuli\OneDrive\Research\geomfum_proj\geomfum\geomfum\_backend\pytorch\sparse.py:22: UserWarning: Sparse CSC tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return _torch.sparse_csc_tensor(ccol_indices, row_indices, values, size=array.shape)


## Further reading

* [How to use ReMatching to compute a functional map?](./13_rematching.ipynb)